### Library

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

### Autoencoder Model

In [6]:
class Autoencoder_Linear_User(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super().__init__()
        
        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, latent_dim)
        )
        
        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, input_dim)
        )

    def forward(self, x):
        latent = self.encoder(x)  # Compress user profile
        reconstructed = self.decoder(latent)  # Reconstruct original profile
        return reconstructed, latent


### Model Initialization

In [7]:
input_dim = 50  # Example: 50 user profile features
latent_dim = 10  # Compress profile into 10 dimensions

In [8]:
model = Autoencoder_Linear_User(input_dim, latent_dim) # Initialize Autoencoder Model
criterion = nn.MSELoss() # Define loss function (reconstruction error)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5) # Adjust weights to minimize loss

### Model Training

In [ ]:
# training
num_epochs = 10
outputs = []
for epoch in range(num_epochs):
    for (img, _) in data_loader:
        img = img.reshape(-1, 28*28)
        recon = model(img)
        loss = criterion(recon, img)

        optimizer.zero_grad() # Reset gradients
        loss.backward() # Compute gradients
        optimizer.step() # Update model parameters

    print(f"Epoch:{epoch+1}, Loss:{loss.item():.4f}")
    outputs.append((epoch, img, recon))

In [ ]:
epochs = 50  # Number of training iterations

for epoch in range(epochs):
    for batch in dataloader:
        batch_data = batch[0]  # Extract batch
        
        optimizer.zero_grad()  # Reset gradients
        reconstructed, latent = autoencoder(batch_data)  # Forward pass
        loss = criterion(reconstructed, batch_data)  # Compute reconstruction loss
        
        loss.backward()  # Compute gradients
        optimizer.step()  # Update model parameters
    
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")
